## Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

## Pydantic
Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [2]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model  = init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000026574CEEE70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026574AC0EC0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title:str = Field(description = "The title of the movie")
    year:int = Field(description = "This year the movie was released")
    director:str = Field(description = "the director of the movie")
    rating:float = Field(description = "The movies rating out of 10")

In [7]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000026574CEEE70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026574AC0EC0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'the director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', '

In [9]:
model.invoke("Rrovide details about the movie house of the Ashes")

AIMessage(content='<think>\nOkay, the user is asking about the movie "House of the Ashes." Let me start by checking if that\'s the correct title. Sometimes people mix up titles, so maybe it\'s a typo. Wait, "House of the Ashes" doesn\'t ring a bell. Let me think. There\'s a movie called "House of the Dead," but that\'s different. Maybe they meant "House of the Dragon," but that\'s a TV series. Or perhaps "House of the Spirits," which is a well-known film. Alternatively, maybe it\'s "The House of the Flying Daggers," a wuxia film. Let me verify.\n\nI should consider possible misspellings. "Ashes" could be part of a different title. Let me search my knowledge base for movies with "Ashes" in the title. There\'s "The Ashes" (2016), but that\'s a documentary. "Ashes and Snow" is a film, but it\'s not widely known. Maybe the user is referring to a different title altogether. Wait, "Ashes to Ashes" is a TV series. Hmm. Alternatively, maybe it\'s "House of Ashes," but I don\'t recall a movie b

In [11]:
response = model_with_structure.invoke("Rrovide details about the movie house of the Ashes")
response

Movie(title='House of the Ashes', year=2008, director='Michael Cuesta', rating=6.5)

### Message output alogside parsed structure

In [13]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description = "The title of the movie")
    year: int = Field(..., description = "The year the movie was released")
    director: str = Field(..., description = "The director of the movie")
    rating: float = Field(..., description = "The movie's rating out of 10")


model_with_structure = model.with_structured_output(Movie, include_raw = True)

response = model_with_structure.invoke("Rrovide details about the movie house of the Ashes")
response


{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie "House of the Ashes." Let me check if I have that information. The available function is called Movie, which requires title, year, director, and rating.\n\nFirst, I need to confirm the exact title. The user wrote "house of the Ashes," but the correct title might be "House of Ashes." I\'ll assume that\'s the right one. Now, I remember that "House of Ashes" is a documentary directed by Ron Franscell. It was released in 2008. The rating is a bit tricky; I think it\'s around 7.2 on IMDb. Let me double-check those details to make sure. Yes, the director is Ron Franscell, year 2008, and the rating is 7.2. I\'ll use the Movie function to provide this information. I need to structure the JSON correctly with the required parameters. Make sure all required fields are included and the types are correct. Number for rating, integer for year, strings for title and director. Alr

## Nested Structure

In [16]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name:str
    role:str
class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres: list[str]
    budget:float | None = Field(None, description = "Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Rrovide details about the movie house of the Ashes")
response

MovieDetails(title='House of the Ashes', year=2008, cast=[Actor(name='Kiefer Sutherland', role='Father'), Actor(name='Shane West', role='Son'), Actor(name='Natalie Portman', role='Daughter')], genres=['Drama', 'Thriller'], budget=25.0)

## TypedDict

TypedDict provides a simpler alternative using Python's built-in typing, ideal when you do'nt need runtime validation

In [17]:
from typing_extensions import TypedDict, Annotated

class Movie(TypedDict):
    """A movie with details"""
    title:Annotated[str,..., "The title of the movie"]
    year:Annotated[int,..., "The year the movie was released"]
    director:Annotated[str,..., "The director of the movie"]
    rating: Annotated[float,..., "The movie's rating out of 10"]

model_with_typeDict = model.with_structured_output(Movie)
response = model_with_typeDict.invoke("Rrovide details about the movie house of the Ashes")
response


{'director': 'Paul Shrader',
 'rating': 6.5,
 'title': 'House of the Ashes',
 'year': 2008}

In [20]:
class Actor(TypedDict):
    name:str
    role:str
class MovieDetails(TypedDict):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float | None = Field(None, description = "Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Rrovide details about the movie house of the Ashes")
response

{'budget': 15000000,
 'cast': [{'name': 'John Doe', 'role': 'Actor 1'},
  {'name': 'Jane Smith', 'role': 'Actress 1'}],
 'genres': ['Drama', 'Thriller'],
 'title': 'House of the Ashes',
 'year': 2023}

## DataClasses
 
A data class is a class typical containing mainly data, althought there aren't really any restrictions, You create it using the @dataclass decorator

In [26]:
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [29]:
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name:str = Field(description = "The name of the person")
    email:str  = Field(desription = "The email address of the person")
    phone: int = Field(description = "The phone number of the person")

agent = create_agent(
    model = "gpt-5",
    response_format = ContactInfo #Auto-selects ProviderStrategy 
)
result = agent.invoke({
    "messages": [{"role":"user", "content": "Extract contact info from: Jhone Dea, oybek07valiyev@gmail.com, (010) 75320407"}]

})
result

{'messages': [HumanMessage(content='Extract contact info from: Jhone Dea, oybek07valiyev@gmail.com, (010) 75320407', additional_kwargs={}, response_metadata={}, id='4ae52dfd-7016-4716-aa5b-fef0dd384daa'),
  AIMessage(content='{"name":"Jhone Dea","email":"oybek07valiyev@gmail.com","phone":1075320407}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 423, 'prompt_tokens': 212, 'total_tokens': 635, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DC1iyZLn1IZYIt8JXo4CguYNRhOag', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c850f-ec6d-7c93-8c42-044e00d420cb-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 